Initialize Google Earth Engine API

In [1]:
import ee

# trigger the authentication flow
ee.Authenticate()

# initialize the library
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=yy36GNo6OnGY7oX9Bwl55vG9-51c9FZWiaiOu8eBS6E&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6k4NOD8q9Q7ksLl8cCazQ8eB-rpFnBLiOr3HmHXFVHKadCmaejANo

Successfully saved authorization token.


Import some python modules and enable inline graphics

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

Make use of interactive maps with the package Folium

In [3]:
# import the Folium library.
import folium

# define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

Define the region of interest with GeoJSON

In [4]:
coords = [-8.49606, 41.49750, -8.36868, 41.59050]

geoJSON = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "type": "Polygon",
        "coordinates": coords
    }
}

aoi = ee.Geometry.Rectangle(coords)

Get the image collection from the Google Earth Engine archives

In [5]:
collection = (ee.ImageCollection('COPERNICUS/S1_GRD')                      # specify image archive
                .filterBounds(aoi)                                         # specify area on interest
                .filterDate(ee.Date('2014-01-01'),ee.Date('2021-01-01'))   # specify time period
                .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')) # specify orbit pass 
                .sort('system:time_start'))                                # sort by date

Get collection as a list and clip all images to the area of interest

In [6]:
# get the image collection as a list
images = collection.toList(collection.size())

print('Number of images available:', images.length().getInfo())

# clip an image to the area of interest
def clip_image(image):
  return ee.Image(image).clip(aoi)

# clip all images to the area of interest
images = ee.List(images.map(clip_image))

Number of images available: 313


Create a RGB composite from the diferent bands. For Sentinel-1 data, the standard way to make the RGB composite is VV for red, VH for green and VV/VH for blue

In [7]:
def to_rgb(image):
  return ee.Image.rgb(image.select('VV'),                            
                      image.select('VH'),                            
                      image.select('VV').divide(image.select('VH')))

Convert a Earth Engine Image into a Pillow Image

In [8]:
from PIL import Image
import requests

def to_pillow(image):
  url = image.getThumbURL({'min': [-20, -20, 0], 'max': [0, 0, 2]})
  return Image.open(requests.get(url, stream=True).raw)

Crop the center of an image

In [9]:
def crop_image(image, new_width, new_height):
  image_width, image_height = image.size
  left = round((image_width - new_width)/2)
  top = round((image_height - new_height)/2)
  x_right = round(image_width - new_width) - left
  x_bottom = round(image_height - new_height) - top
  right = image_width - x_right
  bottom = image_height - x_bottom
  return image.crop((left, top, right, bottom))

Convert a Pillow Image into a Numpy Array

In [14]:
def to_array(image):
  data = np.array(image)
  return data[:, :, 0]

Convert a Numpy Array into a Pilow Image

In [15]:
from numpy import asarray

def to_image(array):
  image = Image.fromarray(array)
  if image.mode != 'RGB':
    image = image.convert('RGB')
  return image

Prepare data

In [53]:
def prepare_data(images):
  nr_images = images.length().getInfo()
  data = []
  #for i in range(nr_images):
  for i in range(10):
    image = ee.Image(images.get(i))
    image_rgb = to_rgb(image)
    image_pillow = to_pillow(image_rgb)
    image_cropped = crop_image(image_pillow, 1024, 1024)
    image_data = to_array(image_cropped)
    data.append(image_data)
  return np.array(data)

data = prepare_data(images)

print(data.shape)

(10, 1024, 1024)


Create set to make the problem supervised

In [54]:
def to_supervised(data):
  nr_images = np.size(data, 0)
  changes = np.empty((nr_images - 1, 1024, 1024))
  for i in range(nr_images - 1):
    ratio = np.subtract(data[i], data[i + 1])
    changes[i] = ratio
  return changes

changes = to_supervised(data)

print(changes.shape)

(9, 1024, 1024)


Normalize pixel values. The images are in gray scale, so the values are between [0, 255]  

In [55]:
def normalize_data(data):
  return data.astype('float32') / 255

data_normalized = normalize_data(data)
changes_normalized = normalize_data(changes)

Slice an array into smaller arrays

In [58]:
def slice_array(array, width, height):
  slices = []
  for x in range(0, array.shape[0], height):
    for y in range(0, array.shape[1], width):
      slices.append(array[x: x + height, y: y + width])
  return np.array(slices)

def slice_arrays(arrays):
  slices = []
  for array in arrays:
    slices.append(slice_array(array, 128, 128))
  return np.array(slices)

data_slices = slice_arrays(data_normalized)
changes_slices = slice_arrays(changes_normalized)

print(data_slices.shape)
print(changes_slices.shape)

(10, 64, 128, 128)
(9, 64, 128, 128)


Split data into training and testing sets

In [59]:
import math

def split_data(data, percentage):
  nr_images, _, _ = data.shape
  train_size = math.floor(nr_images * percentage)
  x_train, x_test = data[:train_size,:], data[train_size:,:]
  y_train, y_test = changes[:(train_size - 1),:], changes[(train_size):,:]
  return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = split_data(data, 0.8)
print('Training data set shape:', x_train.shape)
print('Training changes set shape:', y_train.shape)
print('Testing data set shape:', x_test.shape)
print('Testing changes set shape:', y_test.shape)

Training data set shape: (8, 1024, 1024)
Training changes set shape: (7, 1024, 1024)
Testing data set shape: (2, 1024, 1024)
Testing changes set shape: (1, 1024, 1024)
